# Imports

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from geopy.geocoders import Nominatim

In [4]:
import folium

# Cálculo da latitude e longitude

In [5]:
# Definição
cidades = ['Barcelona',
           'Montserrat',
           'Costa Brava',
           'Cadaqués',
           'Figueres',
           'Girona',
           'Vic e Rupit',
           'Olot',
           'Cap de Creus',
           'Besalú',
           'Garrotxa']
pais = 'Espanha'

In [6]:
lista_cidades = []
lista_latitudes = []
lista_longitudes = []

# Encontrar lat-long de cada cidade
for cidade in cidades:
    
    geolocator = Nominatim(user_agent="carattaolivajulia@gmail.com")

    localizacao = geolocator.geocode(f"{cidade}, {pais}")

    # Verifica se a localização foi encontrada
    if localizacao:
        print(f"{cidade}: {(localizacao.latitude, localizacao.longitude)}")
        lista_cidades.append(cidade)
        lista_latitudes.append(localizacao.latitude)
        lista_longitudes.append(localizacao.longitude)
    else:
        print(f"Coordenadas de {cidade} não encontradas.")
        lista_cidades.append(cidade)
        lista_latitudes.append(None)
        lista_longitudes.append(None)

# DF de lat-long
df_lat_long = pd.DataFrame({'Cidade': lista_cidades, 'Latitude': lista_latitudes, 'Longitude': lista_longitudes})

Barcelona: (41.3828939, 2.1774322)
Montserrat: (39.3576494, -0.6031)
Costa Brava: (41.7037675, 2.9415943)
Cadaqués: (42.2893484, 3.2752159)
Figueres: (42.2666314, 2.9638434)
Girona: (41.9793006, 2.8199439)
Vic e Rupit: (42.07459, 2.4546567)
Olot: (42.1822177, 2.4890211)
Cap de Creus: (42.319456, 3.3223466)
Besalú: (42.1986443, 2.6959559)
Garrotxa: (42.173081100000005, 2.5177403227464943)


In [7]:
df_lat_long

,Cidade,Latitude,Longitude
0,Barcelona,41.382894,2.177432
1,Montserrat,39.357649,-0.603100
2,Costa Brava,41.703767,2.941594
3,Cadaqués,42.289348,3.275216
4,Figueres,42.266631,2.963843
5,Girona,41.979301,2.819944
6,Vic e Rupit,42.074590,2.454657
7,Olot,42.182218,2.489021
8,Cap de Creus,42.319456,3.322347
9,Besalú,42.198644,2.695956


In [8]:
# Mapa centrado na média das coordenadas
mapa = folium.Map(location=[df_lat_long['Latitude'].mean(), 
                            df_lat_long['Longitude'].mean()], 
                            zoom_start=6, 
                            width='80%',  # Define a largura (pode ser percentual ou em pixels, ex: '500px')
                            height='80%'  # Define a altura (pode ser percentual ou em pixels, ex: '300px')
                 )

# Adiciona pontos de cada cidade
for _, row in df_lat_long.iterrows():
    folium.Marker(location=[row['Latitude'], row['Longitude']], popup=row['Cidade']).add_to(mapa)


mapa

# Cálculo da distância no globo

In [9]:
import math

def distancias_globo(lat1, lon1, lat2, lon2):
    
    # Converte latitude e longitude de graus para radianos
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Diferenças entre os pontos
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Fórmula de Haversine
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # Raio da Terra em km (pode mudar para milhas se necessário)
    R = 6371.0
    distance = R * c
    
    return distance

In [10]:
def calcular_distancias_em_df(df):
    
    distancias = []
    
    for i, origem in df.iterrows():
        
        for j, destino in df.iterrows():
            
            if i != j:
                distancia = distancias_globo(origem['Latitude'], origem['Longitude'], destino['Latitude'], destino['Longitude'])
                distancias.append({'Origem': origem['Cidade'], 'Destino': destino['Cidade'], 'Distancia_km': distancia})

    return pd.DataFrame(distancias)

In [11]:
df_distancias_globo = calcular_distancias_em_df(df_lat_long)

In [12]:
df_distancias_globo.sort_values(by='Distancia_km', ascending=True).head()

,Origem,Destino,Distancia_km
107,Garrotxa,Olot,2.575399
79,Olot,Garrotxa,2.575399
37,Cadaqués,Cap de Creus,5.121574
83,Cap de Creus,Cadaqués,5.121574
69,Vic e Rupit,Garrotxa,12.124683


# Cálculo do tempo

In [13]:
df_distancias_globo['tempo_horas'] = round((df_distancias_globo['Distancia_km']/80),2)
df_distancias_globo['tempo_minutos'] = round((df_distancias_globo['Distancia_km']/80)*60,2)

df_distancias_globo.head()

,Origem,Destino,Distancia_km,tempo_horas,tempo_minutos
0,Barcelona,Montserrat,325.849591,4.07,244.39
1,Barcelona,Costa Brava,72.921390,0.91,54.69
2,Barcelona,Cadaqués,135.757113,1.70,101.82
3,Barcelona,Figueres,117.908084,1.47,88.43
4,Barcelona,Girona,85.117740,1.06,63.84


In [14]:
df_distancias_globo['menores_tempos'] = df_distancias_globo.sort_values(by=['Origem', 'tempo_minutos']).groupby('Origem').cumcount() + 1

In [15]:
df_distancias_globo[df_distancias_globo['Origem'] == 'Milão'].sort_values(by='tempo_minutos')

,Origem,Destino,Distancia_km,tempo_horas,tempo_minutos,menores_tempos


# Força bruta: cálculo de melhor rota
- Demora MUITO! Inviável

In [16]:
# import networkx as nx
# import itertools

# # Criar o grafo usando NetworkX
# G = nx.Graph()

# # Adicionar as cidades e as distâncias no grafo
# for _, row in df_distancias_globo.iterrows():
#     G.add_edge(row['Origem'], row['Destino'], weight=row['Distancia_km'])

# # Função para calcular o custo total de uma rota
# def calcular_custo_rota(G, rota):
#     custo_total = 0
#     for i in range(len(rota) - 1):
#         custo_total += nx.shortest_path_length(G, source=rota[i], target=rota[i+1], weight='weight')
#     # Retorna ao ponto inicial para fechar o ciclo
#     custo_total += nx.shortest_path_length(G, source=rota[-1], target=rota[0], weight='weight')
#     return custo_total

# # Função para encontrar a menor rota que passa por todas as cidades
# def menor_rota_tsp(G):
#     cidades = list(G.nodes)
#     menor_rota = None
#     menor_distancia = float('inf')

#     # Gerar todas as permutações possíveis das cidades
#     for rota in itertools.permutations(cidades):
#         distancia_atual = calcular_custo_rota(G, rota)
#         if distancia_atual < menor_distancia:
#             menor_distancia = distancia_atual
#             menor_rota = rota

#     return menor_rota, menor_distancia

In [17]:
# # Encontrar e imprimir a menor rota que passa por todas as cidades

# rota, distancia = menor_rota_tsp(G)
# print(f"A menor rota que passa por todas as cidades é: {rota}, com uma distância de {distancia}")

# Problema do Caixeiro Viajante (TSP): cálculo de melhor rota

## Cálculo da melhor rota

In [18]:
from pulp import *

In [19]:
df_lat_long

,Cidade,Latitude,Longitude
0,Barcelona,41.382894,2.177432
1,Montserrat,39.357649,-0.603100
2,Costa Brava,41.703767,2.941594
3,Cadaqués,42.289348,3.275216
4,Figueres,42.266631,2.963843
5,Girona,41.979301,2.819944
6,Vic e Rupit,42.074590,2.454657
7,Olot,42.182218,2.489021
8,Cap de Creus,42.319456,3.322347
9,Besalú,42.198644,2.695956


In [20]:
# Criar uma matriz de distâncias
n = len(df_lat_long)
matriz_distancia_cidades = np.zeros((n, n))

# Preencher a matriz com as distâncias calculadas
for i in range(n):
    for j in range(n):
        if i != j:
            matriz_distancia_cidades[i][j] = distancias_globo(df_lat_long['Latitude'][i],
                                                              df_lat_long['Longitude'][i], 
                                                              df_lat_long['Latitude'][j], 
                                                              df_lat_long['Longitude'][j])

            cidade_i = df_lat_long['Cidade'][i]
            cidade_j = df_lat_long['Cidade'][j]

            # print(f'\n>> Iteração ({i}, {j}): ({cidade_i}, {cidade_j}). Distância: {matriz_distancia_cidades[i][j]}')

In [21]:
# Exibir a matriz de distâncias
# print(matriz_distancia_cidades)

In [22]:
matriz_distancia_cidades.shape

(11, 11)

In [23]:
n = len(matriz_distancia_cidades)

# Criar loop
rotas_individuais = [(i,j) for i in range(n) for j in range(n) if matriz_distancia_cidades[i,j] != 0.0]

print(rotas_individuais)

[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (1, 0), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 0), (2, 1), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 0), (3, 1), (3, 2), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (4, 0), (4, 1), (4, 2), (4, 3), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 7), (6, 8), (6, 9), (6, 10), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 8), (7, 9), (7, 10), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 9), (8, 10), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 10), (10, 0), (10, 1), (10, 2), (10, 3), (10, 4), (10, 5), (10, 6), (10, 7), (10, 8), (10, 9)]


In [24]:
# Aplicar o caixeiro viajante para minimização da distância total da rota
tsp = LpProblem("RotaCidadesEspanha", LpMinimize)
tsp

RotaCidadesEspanha:
MINIMIZE
None
VARIABLES

In [25]:
# Variáveis de decisão

## Variáveis xij: Binárias e indicam se a rota entre as cidades i e j é incluída na solução.
## Variáveis ui: Contínuas e representam a ordem das cidades para ajudar a prevenir a formação de sub-ciclos.

x = LpVariable.dicts("x", rotas_individuais, cat='Binary')

u = LpVariable.dicts("u", [i for i in range(n)], lowBound=1, upBound=n, cat='Continuous') # Ordem que as rotas individuais são visitadas

In [26]:
# Função Objetivo: 
## Minimizar a soma total das distâncias entre as cidades, multiplicada pela variável binária que indica se a rota é parte da solução ou não
## Portanto, se uma rota é escolhida (x[i,j] = 1), a distância correspondente é adicionada à soma total

# Somar a função objetivo à função já existente no problema

tsp += lpSum([matriz_distancia_cidades[i,j] * x[i,j] for (i,j) in rotas_individuais])
# lpSum: Soma lista de expressões lineares

In [27]:
# Imprimir a função objetivo
print(">> Função Objetivo:\n")
print(tsp.objective)

>> Função Objetivo:

325.84959102523743*x_(0,_1) + 92.28481384831142*x_(0,_10) + 72.92139024769585*x_(0,_2) + 135.75711278089133*x_(0,_3) + 117.90808375476341*x_(0,_4) + 85.11774024414085*x_(0,_5) + 80.2798698007433*x_(0,_6) + 92.55932943026524*x_(0,_7) + 140.84459858676843*x_(0,_8) + 100.37775464944617*x_(0,_9) + 325.84959102523743*x_(1,_0) + 408.6996647943191*x_(1,_10) + 397.1794793805916*x_(1,_2) + 461.1650750873129*x_(1,_3) + 441.2111947962737*x_(1,_4) + 410.2095362234402*x_(1,_5) + 397.03573898886964*x_(1,_6) + 407.9196913217714*x_(1,_7) + 466.2803735524424*x_(1,_8) + 420.595330247397*x_(1,_9) + 92.28481384831142*x_(10,_0) + 408.6996647943191*x_(10,_1) + 62.86798549347136*x_(10,_2) + 63.6909344365378*x_(10,_3) + 38.17996443343128*x_(10,_4) + 32.96071684031254*x_(10,_5) + 12.124682967570482*x_(10,_6) + 2.575398872082858*x_(10,_7) + 68.2001875065139*x_(10,_8) + 14.95615995735821*x_(10,_9) + 72.92139024769585*x_(2,_0) + 397.1794793805916*x_(2,_1) + 62.86798549347136*x_(2,_10) + 70.70

In [28]:
# Restrição 1: Garantir que cada cidade seja visitada apenas uma vez
for j in range(n):
    tsp += lpSum([x[i,j] for (i,m) in rotas_individuais if m==j]) == 1

# Restrição 2: Garantir que cada cidade seja partida exatamente uma vez
for i in range(n):
    tsp += lpSum([x[i,j] for (m,j) in rotas_individuais if m==i]) == 1

# Restrição 3: Eliminar subciclos
for (i,j) in rotas_individuais:
    if i > 0 and i != j:
        tsp += u[i] - u[j] + n*x[i,j] <= n-1

In [29]:
# Resolver o modelo
resolver_modelo = tsp.solve()
print(f'Status do problema: {LpStatus[resolver_modelo]}')

Status do problema: Optimal


In [30]:
# Inicializa uma lista para armazenar as variáveis
variaveis_selecionadas = []

for var in tsp.variables():
    if var.varValue > 0 and var.name.startswith('x'):
        variaveis_selecionadas.append([var.name])

# Cri DF de ordem
df_ordem_cidades = pd.DataFrame(variaveis_selecionadas, columns=['Ordem'])

In [31]:
# Extrair destino e origem
df_ordem_cidades[['ponto_origem', 'ponto_destino']] = df_ordem_cidades['Ordem'].str.extract(r'\((\d+),_(\d+)\)')
df_ordem_cidades['ponto_origem'] = df_ordem_cidades['ponto_origem'].astype(int)
df_ordem_cidades['ponto_destino'] = df_ordem_cidades['ponto_destino'].astype(int)

# Criar de-para de index
df_de_para = df_lat_long.reset_index().rename(columns={'index': 'index'})

# Usa a função map para substituir os números pelos nomes das cidades
df_ordem_cidades['Cidade_Origem'] = df_ordem_cidades['ponto_origem'].map(df_de_para.set_index('index')['Cidade'])
df_ordem_cidades['Cidade_Destino'] = df_ordem_cidades['ponto_destino'].map(df_de_para.set_index('index')['Cidade'])

In [32]:
df_ordem_cidades

,Ordem,ponto_origem,ponto_destino,Cidade_Origem,Cidade_Destino
0,"x_(0,_2)",0,2,Barcelona,Costa Brava
1,"x_(1,_0)",1,0,Montserrat,Barcelona
2,"x_(10,_7)",10,7,Garrotxa,Olot
3,"x_(2,_5)",2,5,Costa Brava,Girona
4,"x_(3,_8)",3,8,Cadaqués,Cap de Creus
5,"x_(4,_9)",4,9,Figueres,Besalú
6,"x_(5,_3)",5,3,Girona,Cadaqués
7,"x_(6,_1)",6,1,Vic e Rupit,Montserrat
8,"x_(7,_6)",7,6,Olot,Vic e Rupit
9,"x_(8,_4)",8,4,Cap de Creus,Figueres


In [33]:
# Inicializar para armazenar
ordem_cidades = []
visitadas = set() 

# Ponto de partida
cidade_atual = df_ordem_cidades['Cidade_Origem'].iloc[0] 
cidade_destino = df_ordem_cidades['Cidade_Destino'].iloc[0]

# Loop
while cidade_atual and cidade_atual not in visitadas:

    # Registra ponto atual
    ordem_cidades.append(cidade_atual)
    visitadas.add(cidade_atual)

    # Atualiza ponto atual (origem)
    index_proxima_cidade_origem = df_ordem_cidades[df_ordem_cidades['Cidade_Origem'] == cidade_destino].index[0]
    cidade_atual = df_ordem_cidades['Cidade_Origem'].iloc[index_proxima_cidade_origem] 
    cidade_destino = df_ordem_cidades['Cidade_Destino'].iloc[index_proxima_cidade_origem]

# Cria uma lista de resultados 
resultado = [f'{i + 1}: {cidade}' for i, cidade in enumerate(ordem_cidades)]
print(f'Ordem de visita:\n')
for r in resultado:
    print(r)

# Cria DFs
df_ordem = pd.DataFrame([r.split(': ') for r in resultado], columns=['Ordem de visita', 'Cidade'])
df_aux_1 = pd.merge(df_ordem, df_lat_long, on='Cidade', how='left')

Ordem de visita:

1: Barcelona
2: Costa Brava
3: Girona
4: Cadaqués
5: Cap de Creus
6: Figueres
7: Besalú
8: Garrotxa
9: Olot
10: Vic e Rupit
11: Montserrat


In [34]:
def adicionar_distancias_proximo_ponto(df):

    distancias = []
    
    for i in range(len(df) - 1):
        origem = df.iloc[i]
        destino = df.iloc[i + 1]
        
        distancia = distancias_globo(origem['Latitude'], origem['Longitude'], destino['Latitude'], destino['Longitude'])
        distancias.append(distancia)
    
    # Para a última cidade, não há próximo ponto, então adicionamos NaN ou 0
    distancias.append(np.nan)  
    
    df['Distancia_ate_Proximo_Ponto_km'] = distancias
    
    return df

In [35]:
df_final = adicionar_distancias_proximo_ponto(df_aux_1)

df_final['Distancia_ate_Proximo_Ponto_km'] = df_final['Distancia_ate_Proximo_Ponto_km'].round(4)
df_final['tempo_horas'] = round((df_final['Distancia_ate_Proximo_Ponto_km']/80),2)
df_final['tempo_minutos'] = round((df_final['Distancia_ate_Proximo_Ponto_km']/80)*60,2)

df_final

,Ordem de visita,Cidade,Latitude,Longitude,Distancia_ate_Proximo_Ponto_km,tempo_horas,tempo_minutos
0,1,Barcelona,41.382894,2.177432,72.9214,0.91,54.69
1,2,Costa Brava,41.703767,2.941594,32.2527,0.40,24.19
2,3,Girona,41.979301,2.819944,50.9698,0.64,38.23
3,4,Cadaqués,42.289348,3.275216,5.1216,0.06,3.84
4,5,Cap de Creus,42.319456,3.322347,30.0670,0.38,22.55
5,6,Figueres,42.266631,2.963843,23.3151,0.29,17.49
6,7,Besalú,42.198644,2.695956,14.9562,0.19,11.22
7,8,Garrotxa,42.173081,2.517740,2.5754,0.03,1.93
8,9,Olot,42.182218,2.489021,12.2986,0.15,9.22
9,10,Vic e Rupit,42.074590,2.454657,397.0357,4.96,297.78


In [36]:
# Inicializar o mapa
mapa = folium.Map(location=[df_lat_long['Latitude'].mean(), df_lat_long['Longitude'].mean()], zoom_start=6, width='80%', height='80%') 

coordenadas = []

for i, cidade in enumerate(ordem_cidades):
    lat_long = df_lat_long[df_lat_long['Cidade'] == cidade]
    latitude = lat_long['Latitude'].values[0]
    longitude = lat_long['Longitude'].values[0]
    
    coordenadas.append([latitude, longitude])
    
    folium.Marker(
        location=[latitude, longitude], 
        popup=f"{i + 1}: {cidade}",
        icon=folium.DivIcon(html=f"""
            <div style="font-size: 8pt; color : black; 
                        background-color: white; border-radius: 50%; 
                        padding: 5px; text-align: center; width: 20px; height: 20px;">
                {i + 1}
            </div>"""),
        tooltip=f"{i + 1}: {cidade}" 
    ).add_to(mapa)

folium.PolyLine(coordenadas, color="blue",weight=5, opacity=0.7).add_to(mapa)

mapa